In [ ]:
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from time import sleep
import time
from tqdm import tqdm

# Set the account and date range
account = "Underdog__NBA"
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

usernames = ["Go0sed"]
passwords = ["Baseball$3"]
num_of_tweets = 75

# Convert dates to string format 'YYYY-MM-DD'
end_date_str = end_date.strftime('%Y-%m-%d')
start_date_str = start_date.strftime('%Y-%m-%d')

UserTags, TimeStamps, Tweets_url = [], [], []

for user, pw in zip(usernames, passwords):
    driver = webdriver.Chrome()

    driver.get(f'https://twitter.com/search?q=(from%3A{account})%20until%3A{end_date_str}%20since%3A{start_date_str}&src=typed_query&f=live')

    # Wait for the page to load before continuing
    sleep(7)

    #### LOGIN ######
    # Find the username input field using its XPATH and enter a username
    username = driver.find_element(By.XPATH, "//input[@name='text']")
    username.send_keys(user)

    # Find the 'Next' button using its XPATH and click it to move to the password field
    next_button = driver.find_element(By.XPATH, "//span[contains(text(),'Next')]")
    next_button.click()

    # Wait for the next page to load before continuing
    sleep(10)

    # Find the password input field using its XPATH and enter a password
    password = driver.find_element(By.XPATH, "//input[@name='password']")
    password.send_keys(pw)

    # Find the 'Log in' button using its XPATH and click it to log in
    log_in = driver.find_element(By.XPATH, "//span[contains(text(),'Log in')]")
    log_in.click()

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
    articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")

    Tweets2, Tweets = [], []

    begin = time.time()

    with tqdm(total=num_of_tweets, desc="Extracting tweets", unit="tweet") as pbar:
        while True:
            for article in articles:
                try:
                    Tweet = article.find_element(By.XPATH, ".//div[@data-testid='tweetText']").text
                    if ' out' in Tweet.lower() or 'miss rest of season' in Tweet.lower() or 'indefinitely' in Tweer.lower() or 'restriction' in Tweet.lower() or 'late scratch' in Tweet.lower() or ' doubtful' in Tweet.lower():
                        Tweets.append(Tweet)

                        UserTag = article.find_element(By.XPATH, ".//div[@data-testid='User-Name']").text
                        UserTags.append(UserTag)

                        TimeStamp = article.find_element(By.XPATH, ".//time").get_attribute('datetime')
                        TimeStamps.append(TimeStamp)

                        Tweets_url.append((Tweet, TimeStamp.split('T')[0]))
                        pbar.update(1)
                except:
                    pass

            Tweets2 = list(set(Tweets))

            if len(Tweets2) >= num_of_tweets:
                print(len(Tweets2))
                break

            driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
            time.sleep(3)
            articles = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")

    end = time.time()
    print(f"Scraping done for username: {user}")
    print("Time taken: ", end - begin)

    driver.close()

# Convert the collected data to a DataFrame and save it to a CSV file
tweet, ts = zip(*Tweets_url)
df = pd.DataFrame({"tweetContent": tweet, "timeStamp": ts})
df.to_csv(r'C:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\Underdog_Scrape.csv', index=False)